<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/StabilityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift, bode

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

import sympy
from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig
from scipy.integrate import solve_ivp
from scipy.optimize import root, minimize

In [3]:
Ac = 0.9
b = 0.29
Kc=0.3
qin_ss = 1.12
h_initial_ss = np.full(3, qin_ss/b)
h3sp_initial = h_initial_ss[-1]


def rhs(t, v, h3sp):
    h1, h2, h3 = v
    err = h3sp  - h3
    q12 = b*h1
    q23 = b*h2
    q3 = b*h3
    qin1 = qin_ss + Kc*err

    dh1  = (qin1 - q12)/Ac
    dh2 = (q12 - q23)/Ac
    dh3 = (q23 - q3 )/Ac
    return jnp.array([dh1, dh2, dh3])

In [4]:
tend=100
res=solve_ivp(lambda t, v: rhs(t, v, h3sp_initial+1.), (0,tend ), h_initial_ss, method='Radau', dense_output=True)
res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.190e-01 ...  6.876e+01  1.000e+02]
        y: [[ 3.862e+00  3.901e+00 ...  4.370e+00  4.371e+00]
            [ 3.862e+00  3.863e+00 ...  4.370e+00  4.371e+00]
            [ 3.862e+00  3.862e+00 ...  4.371e+00  4.371e+00]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x78fd12a42cb0>
 t_events: None
 y_events: None
     nfev: 86
     njev: 1
      nlu: 22

In [5]:
tplot = np.linspace(0,tend,500)
fig=make_subplots()
# fig.add_scatter(x=t,y=h3_dev+h_initial_ss[2], name='linear')
fig.add_scatter(x=tplot, y= res.sol(tplot)[2], name='solve_ivp')
fig.update_layout(width=600, height=400, template='plotly_dark')

In [6]:
K=1/b
tau = Ac/b
Kc*K/(1+Kc*K)

0.5084745762711865

In [7]:
res.sol(tplot)[2][-1] - res.sol(tplot)[2][0]

0.5084768590754143

In [8]:
w = np.logspace(-2,2,1000)

In [9]:
RI=Kc*K/((tau*w*1j +1)**3)
R = np.real(RI)
I = np.imag(RI)

In [10]:
Gp =Kc*K/((tau*s + 1)**3)
t, y = sim(Gp, lambda t: np.sin(1.*t), N=1000, dt=0.1)

In [11]:
fig3=make_subplots()
fig3.add_scatter(x=t, y= y)
fig3.update_layout(width=600, height=400, template='plotly_dark')

In [12]:
AR=np.abs(RI)

In [13]:
phase = np.unwrap(np.arctan2(I,R))*180/np.pi

In [14]:
fig2 = make_subplots(rows=2,cols=1)
fig2.add_scatter(x=w, y=AR, row=1, col=1)
fig2.update_xaxes(type="log", row=1, col=1)
fig2.update_yaxes(type='log', row=1,col=1)
fig2.add_scatter(x=w, y=phase, row=2, col=1)
fig2.update_xaxes(type="log", row=2, col=1)
fig2.update_layout(height=600, width=400, template='plotly_dark', showlegend=False,
                   yaxis2=dict(dtick=60))

In [15]:

K=1.
tau = 10.
z = 0.2

In [16]:
RI = K/((tau**2)*((w*1j)**2) + 2*z*tau*(w*1j) + 1.)
R = np.real(RI)
I = np.imag(RI)

In [17]:
AR = np.abs(RI)
phase = np.unwrap(np.arctan2(I,R))*180/np.pi

In [18]:
fig2 = make_subplots(rows=2,cols=1)
fig2.add_scatter(x=w, y=AR, row=1, col=1)
fig2.update_xaxes(type="log", row=1, col=1)
fig2.update_yaxes(type='log', row=1,col=1)
fig2.add_scatter(x=w, y=phase, row=2, col=1)
fig2.update_xaxes(type="log", row=2, col=1)
fig2.update_layout(height=600, width=400, template='plotly_dark', showlegend=False,
                   yaxis2=dict(dtick=60))

In [19]:
w = np.logspace(-2,1.5,1000)
Kc = 2.
K=1.
tau = 10.
RI = Kc*K * np.exp(-0.1*w*(1j))/(tau*w*(1j) + 1)
R = np.real(RI)
I = np.imag(RI)
AR = np.abs(RI)
phase = np.unwrap(np.arctan2(I,R))*180/np.pi

In [20]:
fig2 = make_subplots(rows=2,cols=1)
fig2.add_scatter(x=w, y=AR, row=1, col=1)
fig2.update_xaxes(type="log", row=1, col=1)
fig2.update_yaxes(type='log', row=1,col=1)
fig2.add_scatter(x=w, y=phase, row=2, col=1)
fig2.update_xaxes(type="log", row=2, col=1)
fig2.update_layout(height=600, width=400, template='plotly_dark', showlegend=False,
                   yaxis2=dict(dtick=60))

In [21]:
PA, PB = 0.586, -0.916
IA, IB = 1.03, -0.165

In [22]:
K=1.
tau=1.
# Kc = 0.5
# taui = 0.1
theta= 0.1
Y = PA*(theta/tau)**PB
Kc = Y/K
taui = tau/(IA + IB*(theta/tau))


Gp = exp(-theta*s)*K/(tau*s+1)
Gc = Kc*(1+1/(taui*s))
Gm = 1.
Gfb = Gc*Gp/(1+Gp*Gc*Gm)
Gfb=simplify(Gfb)

In [23]:
_, AR, phase = bode(Gc*Gp*Gm,w)
fig5 = make_subplots(rows=2,cols=1)
fig5.add_scatter(x=w, y=AR, row=1, col=1)
fig5.update_xaxes(type="log", row=1, col=1)
fig5.update_yaxes(type='log', row=1,col=1)
fig5.add_scatter(x=w, y=phase, row=2, col=1)
fig5.update_xaxes(type="log", row=2, col=1)
fig5.update_layout(height=600, width=400, template='plotly_dark', showlegend=False,
                   yaxis2=dict(dtick=30, range=(-270,0)))

In [24]:
t,y=sim(Gfb, lambda t: 1., N=100, dt=0.1)

In [25]:
fig4=make_subplots()
fig4.add_scatter(x=t, y= y)
fig4.update_layout(width=600, height=400, template='plotly_dark')